In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree

In [30]:
## Read in the entire data set just for eyeballing.
pd_data = pd.read_csv('K_data.csv', header=None)
np_data = np.array(pd_data.values, dtype=np.float32) ## float32 might be better if using a GPU
pd_data

,0,1,2,3,4,5
0,1.729100e-03,1.509,0.52360,3,2,0.0100
1,1.965300e-03,1.558,0.54105,3,2,0.0100
2,2.224300e-03,1.607,0.55851,3,2,0.0100
3,2.507300e-03,1.656,0.57596,3,2,0.0100
4,2.815600e-03,1.705,0.59341,3,2,0.0100
5,3.150700e-03,1.753,0.61087,3,2,0.0100
6,3.513700e-03,1.802,0.62832,3,2,0.0100
7,3.906100e-03,1.850,0.64577,3,2,0.0100
8,4.329200e-03,1.898,0.66323,3,2,0.0100
9,4.784300e-03,1.946,0.68068,3,2,0.0100


In [31]:
## Read in the training data, which includes both the input features and the outputs K.
pd_train = pd.read_csv('K_training_random_stand.csv', header=None)
np_train = np.array(pd_train.values, dtype=np.float32) ## float32 might be better if using a GPU
np_train

array([[2.6582001e-04, 9.4499999e-01, 9.3334609e-01, ..., 1.0000000e+00,
        1.0000000e+00, 2.0000000e-01],
       [6.9525000e-03, 2.1370001e+00, 2.1666348e-01, ..., 0.0000000e+00,
        1.0000000e+00, 5.0000000e-01],
       [5.4947002e-12, 1.1000000e-02, 0.0000000e+00, ..., 1.0000000e+00,
        9.0909094e-02, 6.0000002e-01],
       ...,
       [1.3298000e-10, 2.5000000e-02, 6.6663486e-01, ..., 1.0000000e+00,
        9.0909094e-02, 9.9179999e+02],
       [2.5088999e-16, 1.0000000e-03, 9.1663486e-01, ..., 1.0000000e+00,
        0.0000000e+00, 9.9600000e+02],
       [1.9636000e-06, 2.7700001e-01, 3.6666349e-01, ..., 0.0000000e+00,
        0.0000000e+00, 9.9640002e+02]], dtype=float32)

In [32]:
## Extract the input features: theta, m, n, h_star
features = np_train[:, 2:6]
features

array([[0.9333461 , 1.        , 1.        , 1.        ],
       [0.21666348, 0.        , 0.        , 1.        ],
       [0.        , 1.        , 1.        , 0.09090909],
       ...,
       [0.66663486, 1.        , 1.        , 0.09090909],
       [0.91663486, 1.        , 1.        , 0.        ],
       [0.3666635 , 0.        , 0.        , 0.        ]], dtype=float32)

In [33]:
## Extract the correct outputs K
correct_outputs = np_train[:, 1]
correct_outputs[1:5]

array([2.137, 0.011, 0.03 , 0.022], dtype=float32)

In [77]:
regr_1 = tree.DecisionTreeRegressor(max_depth=5)
regr_1.fit(features, correct_outputs)

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [84]:
## Read in the test data, which includes both the input features and the outputs K.
pd_test = pd.read_csv('K_testing_stand.csv', header=None)
np_test = pd_test.values
np_test_features = np_test[:, 2:6]

In [85]:
np_test_outputs = np_test[:, 1]
np_test_outputs

array([1.509e+00, 1.915e+00, 9.540e-01, 5.470e-01, 2.140e-01, 3.100e-02,
       1.790e-01, 2.200e-02, 1.000e-03])

In [86]:
regr_1.predict(np_test_features)

array([1.84907693e+00, 2.14966667e+00, 9.43624996e-01, 6.35214286e-01,
       2.55612902e-01, 2.62903227e-02, 1.99230768e-01, 2.55625001e-02,
       1.00000005e-03])